In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from PIL import Image
import numpy as np
import csv
import pandas as pd
import keras.utils as image
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

LETTERSTR = "0123456789abcdefghijklmnopqrstuvwxyz"


def toonehot(text):
    labellist = []
    for letter in text:
        onehot = np.array([0 for _ in range(36)])
        num = LETTERSTR.find(letter)
        onehot[num] = 1
        labellist.append(onehot)
    return np.array(labellist)

In [ ]:
# Create CNN Model
print("Creating CNN model...")
in_ = Input((72, 96, 3))
out = in_
out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Flatten()(out)
out = Dropout(0.3)(out)
out = [Dense(36, name='digit1', activation='softmax')(out),\
    Dense(36, name='digit2', activation='softmax')(out),\
    Dense(36, name='digit3', activation='softmax')(out),\
    Dense(36, name='digit4', activation='softmax')(out)]
model = Model(inputs=in_, outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])

In [ ]:
train_data = []
train_label = [[] , [] , [] , []]
path = '/home/undergrad/.kaggle/train/task3'
df = pd.read_csv('/home/undergrad/.kaggle/train/annotations.csv')
for filename in os.listdir(path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img = image.load_img(os.path.join(path , filename) , target_size = (100 , 100))
        aug_images = []
        img = np.expand_dims(img , 0)
        gen = ImageDataGenerator(
        rotation_range = 10 , 
        width_shift_range = 0.15 , 
        height_shift_range = 0.15 , 
        shear_range = 0.15 , 
        zoom_range = 0.1 , 
        channel_shift_range = 10 , 
        )
        aug_iter = gen.flow(img)
        aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(20)]
        label = df.loc[df['filename'] == os.path.join('task3' , filename) , 'label'].item()
        label_ = toonehot(label)
        for aug_pic in np.array(aug_images):
            aug_pic = aug_pic.astype('float')/255.0
            train_data.append(aug_pic)
            for i in range(4):
                train_label[i].append(label_[i])
train_data = np.array(train_data)
train_label = [arr for arr in np.asarray(train_label)]
# print(train_data.shape)
# print(train_label.shape)
# train_data = tf.stack(train_data)
# train_label = tf.stack(train_label)


model.fit(train_data, train_label, batch_size = 400, epochs = 20, verbose=1 , validation_split=0.2) 
model.save("part3_model.h5")         